In [1]:
import re
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import requests
import json
import time
from typing import List, Protocol, TypeVar, Union
from datetime import datetime, timedelta

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from chromadb import Documents, EmbeddingFunction, Embeddings

from tqdm.notebook import tqdm
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')
sys.path.append('../data')
from parsing.parser import parse_pdf
from embeddings import CustomEmbeddings, TogetherEmbeddings, YandexEmbeddings

import arxiv
import pickle

## Download PDFs from ArXiv

In [128]:
papers_ = arxiv.Client().results(arxiv.Search(query='text mining'))
N_PAPERS = 50
save_dir = "./data/raw/text_mining/articles"
downloaded_files = [f for f in listdir(save_dir) if f.endswith('.pdf')]

for i, paper in tqdm(enumerate(papers_), total=N_PAPERS):
    paper = next(papers_)
    
    if f'{paper.title}.pdf' not in downloaded_files:
        try:
            paper.download_pdf(dirpath=save_dir, filename=f'{paper.title}.pdf')
            downloaded_files.append(paper.title + '.pdf')
        except OSError as e:
            print(f'Gor error downloading paper "{paper.title}":\n{e}')
            
    if i == N_PAPERS - 1:
        break

  0%|          | 0/50 [00:00<?, ?it/s]

Gor error downloading paper "Can You Explain That, Better? Comprehensible Text Analytics for SE Applications":
[Errno 22] Invalid argument: './data/raw/text_mining/articles\\Can You Explain That, Better? Comprehensible Text Analytics for SE Applications.pdf'
Gor error downloading paper ""In vivo" spam filtering: A challenge problem for data mining":
[Errno 22] Invalid argument: './data/raw/text_mining/articles\\"In vivo" spam filtering: A challenge problem for data mining.pdf'


## Parse text from Arxiv PDFs

In [3]:
DATADIR = 'data/raw/text_mining/articles/'
PARSED_FILES_DIR = 'data/processed/texts/'

filenames = os.listdir(DATADIR)

for i, filename in enumerate(filenames):
    try:
        print(f'Process file {i + 1} of {len(filenames)}: {filename}...')
        res = parse_pdf(DATADIR + filename,
                        conf_threshold=0.55,
                        verbose=False,
                        debug=False,
                        debug_dir=PARSED_FILES_DIR)
        text = '\n'.join([r[0] for r in res])
        
        name = filename.split('/')
        with open(f'data/processed/texts/{filename.removesuffix("pdf") + "txt"}', 'w', errors='ignore') as f:
            f.write(text)
    except Exception as e:
        print(f'File {filename} failed to process..')
        print(e)

Process file 1 of 57: A Mining-Based Compression Approach for Constraint Satisfaction Problems.pdf...
Statistics dictionary not passed, calculate statistics...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.47it/s]


Statistics calculated.
Results:
{'min_length': 50, 'style': 'Standart', 'size': 10, 'font': 'UCJIPT+CMR10', 'mean_length': 205.66153846153847, 'width': {'hist': array([14,  9,  4,  4,  9,  9,  7,  3,  5, 66], dtype=int64), 'bin_edges': array([  2.75965128,  38.35765528,  73.95565928, 109.55366328,
       145.15166728, 180.74967128, 216.34767528, 251.94567928,
       287.54368328, 323.14168728, 358.73969128])}, 'x0': {'hist': array([ 5,  0, 80, 20, 10,  6,  2,  4,  2,  1], dtype=int64), 'bin_edges': array([ 16.34    ,  59.429976, 102.519952, 145.609928, 188.699904,
       231.78988 , 274.879856, 317.969832, 361.059808, 404.149784,
       447.23976 ])}, 'x1': {'hist': array([ 5,  0,  1,  9,  4,  2, 15, 14,  9, 71], dtype=int64), 'bin_edges': array([ 36.34      ,  83.55595313, 130.77190626, 177.98785938,
       225.20381251, 272.41976564, 319.63571877, 366.8516719 ,
       414.06762502, 461.28357815, 508.49953128])}}


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.37it/s]


Process file 2 of 57: A Survey on Web Multimedia Mining.pdf...
Statistics dictionary not passed, calculate statistics...


...

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00, 11.64it/s]


Process file 57 of 57: Using compression to identify acronyms in text.pdf...
Statistics dictionary not passed, calculate statistics...


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  6.19it/s]


Statistics calculated.
Results:
{'min_length': 50, 'style': 'Standart', 'size': 11, 'font': 'DQPICQ+CMR10', 'mean_length': 152.1809523809524, 'width': {'hist': array([78, 20,  3,  9, 11,  7,  5,  2,  5, 70], dtype=int64), 'bin_edges': array([  3.0218207 ,  47.46328297,  91.90474523, 136.3462075 ,
       180.78766976, 225.22913203, 269.67059429, 314.11205656,
       358.55351882, 402.99498109, 447.43644336])}, 'x0': {'hist': array([  8, 110,  20,  23,   6,  13,  11,   2,  14,   3], dtype=int64), 'bin_edges': array([ 16.34  ,  65.4275, 114.515 , 163.6025, 212.69  , 261.7775,
       310.865 , 359.9525, 409.04  , 458.1275, 507.215 ])}, 'x1': {'hist': array([ 8, 19, 18, 10,  9, 20, 20, 14, 12, 80], dtype=int64), 'bin_edges': array([ 36.34      ,  86.20958434, 136.07916867, 185.94875301,
       235.81833734, 285.68792168, 335.55750601, 385.42709035,
       435.29667468, 485.16625902, 535.03584336])}}


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  7.37it/s]


## Split text to chunks

In [2]:
def get_pdf_text(pdf_filename):
    text = ""
    pdf_reader = PdfReader(pdf_filename)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


def process_text(text: str) -> str:
    text = re.sub(r'\[.*?\]', '', text)
    text = text.replace('>>>', '\n>>>')
    return text


def validate_text(text: str) -> bool:    
    DIGITS_LETTERS_THRESHOLD = 0.5
    n_digits = sum([sym.isdigit() for sym in text])
    n_letters = sum([sym.isalpha() for sym in text])
    
    if n_digits / (n_letters + 1) > DIGITS_LETTERS_THRESHOLD:
        return False
    
    # Оглавления
    if re.search(r' {10,}|(?: \.){10,}', text):
        return False
    
    # Ссылки
#     if re.search(r'www\.|https?://', text):
#         return False
    return True

In [3]:
PARSED_TEXTS_DIR = '../data/processed/texts/'
filepaths = [PARSED_TEXTS_DIR + file for file in os.listdir(PARSED_TEXTS_DIR) if file.endswith('.txt')]

In [4]:
chink_size = 1500

text_splitter = CharacterTextSplitter(
        separator=" ",
        chunk_size=chink_size,
        chunk_overlap=400,
        length_function=len
    )

processed_docs = []

for filename in filepaths:
    with open(filename) as f:
        text = f.read()
        
    valid_chunks = 0
    text_chunks = text_splitter.split_text(text)
        
    for j, chunk in enumerate(text_chunks):            
        chunk = process_text(chunk)
        if len(chunk) > chink_size:
            continue
            
        if validate_text(chunk):
            valid_chunks += 1
            doc = Document(page_content=chunk, metadata=dict(docname=filename.split(sep='/')[-1].removesuffix('.txt')))
            processed_docs.append(doc)

    print(f'Text processed. Got {valid_chunks} texts. Validity is {(valid_chunks / len(text_chunks) * 100):.2f}%')

Text processed. Got 21 texts. Validity is 100.00%
Text processed. Got 34 texts. Validity is 100.00%
Text processed. Got 17 texts. Validity is 100.00%
Text processed. Got 31 texts. Validity is 100.00%
Text processed. Got 38 texts. Validity is 100.00%
Text processed. Got 12 texts. Validity is 100.00%
Text processed. Got 989 texts. Validity is 99.90%
Text processed. Got 131 texts. Validity is 98.50%
Text processed. Got 27 texts. Validity is 100.00%
Text processed. Got 31 texts. Validity is 100.00%
Text processed. Got 24 texts. Validity is 100.00%
Text processed. Got 44 texts. Validity is 100.00%
Text processed. Got 34 texts. Validity is 100.00%
Text processed. Got 20 texts. Validity is 100.00%
Text processed. Got 48 texts. Validity is 100.00%
Text processed. Got 46 texts. Validity is 100.00%
Text processed. Got 76 texts. Validity is 100.00%
Text processed. Got 55 texts. Validity is 100.00%
Text processed. Got 32 texts. Validity is 100.00%
Text processed. Got 31 texts. Validity is 100.00%


Created a chunk of size 16369, which is longer than the specified 1500


Text processed. Got 630 texts. Validity is 98.44%
Text processed. Got 43 texts. Validity is 100.00%
Text processed. Got 344 texts. Validity is 95.56%
Text processed. Got 42 texts. Validity is 100.00%
Text processed. Got 32 texts. Validity is 100.00%
Text processed. Got 25 texts. Validity is 100.00%
Text processed. Got 9 texts. Validity is 100.00%
Text processed. Got 13 texts. Validity is 100.00%
Text processed. Got 241 texts. Validity is 100.00%
Text processed. Got 16 texts. Validity is 100.00%
Text processed. Got 26 texts. Validity is 100.00%


In [5]:
print(f'Parsed {len(processed_docs)} documents.')

Parsed 5471 documents.


## Translate documents

In [6]:
class Translator(Embeddings):  
    def __init__(self, folder_id, oauth_token):
        self.folder_id = folder_id
        self.oauth_token = oauth_token
        
        self._iam_expires_at = None
        self._iam_token = None
        
        self._update_iam_token()
        
        self._url = "https://translate.api.cloud.yandex.net/translate/v2/translate"
        
        self.batch_size = 5

    
    def _update_iam_token(self):
        print('Updating Yandex IAM token.')
        query = {
            "yandexPassportOauthToken": self.oauth_token
        }
        response = requests.post("https://iam.api.cloud.yandex.net/iam/v1/tokens", data=json.dumps(query))
        if response.status_code != 200:
            print(f"Could not update Yandex IAM token. Status code: {response.status_code} Body:\n{response.text}")
            raise Exception('Failed to get Yandex IAM token!')

        response = response.json()
        self._iam_expires_at = datetime.strptime(response['expiresAt'].split('.')[0], "%Y-%m-%dT%H:%M:%S")
        self._iam_token = response['iamToken']
        print(f'IAM token succesully updated: {self._iam_token[:15]} Expires at: {self._iam_expires_at}')
    
    
    def __call__(self, texts: List[str], source='en', target='ru') -> List[str]:
        return self.translate(texts=texts, source=source, target=target)
    
    
    def translate(self, texts: List[str], source='en', target='ru') -> List[str]:
        headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self._iam_token}",
                "x-folder-id": f"{self.folder_id}"
            }
        
        translated_texts = []
        
        for text in tqdm(range(0, len(texts), self.batch_size)):
            if datetime.utcnow() + timedelta(hours=1) > self._iam_expires_at:
                self._update_iam_token()
                headers['Authorization'] = f"Bearer {self._iam_token}"
                
            body = {
                "folderId": self.folder_id,
                "texts": texts[i: i + self.batch_size],
                "targetLanguageCode": target
            }

            retries = 0
            while retries < 5:
                try:
                    response = requests.post(self._url, json=body, headers=headers)
                    
                    if response.status_code == 200:
                        translated_texts += [translation['text'] for translation in response.json()['translations']]
                        break
                    else:
                        time.sleep(1)
                        retries += 1
                except Exception as e:
                    print(e)
                    print(response.text)
            else:
                raise Exception(f'Failed to translate text. Status code: {response.status_code}\nMessage:{response.text}')
        return translated_texts

In [7]:
OAUTH_TOKEN = 'y0_AgAAAABFOtKIAATuwQAAAAEAcNmeAABjadeGLERHyJNjoJ7uwRjlQ_oQjA'
FOLDER_ID = 'b1g6hkb495v635sf93ir'

translator = Translator(FOLDER_ID, OAUTH_TOKEN)

Updating Yandex IAM token.
IAM token succesully updated: t1.9euelZrPx5vN Expires at: 2024-04-07 07:36:53


In [8]:
en_texts = [doc.page_content for doc in processed_docs]

In [9]:
ru_texts = translator(en_texts)

  0%|          | 0/697 [00:00<?, ?it/s]

Exception: Failed to translate text. Status code: 429
Message:{
 "code": 8,
 "message": "limit on units was exceeded. Limit: 1000000, Interval: 1h0m0s",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "ea32ed77-30c3-4a41-a527-01eea4ac04b8"
  }
 ]
}


## Create storage

In [6]:
model = 'BAAI/bge-base-en-v1.5'
token = '4fbc8ea1ec756dd6d645a7db75ebb6e8a85cdaf81b09acb2bbe3143a017d11c8'

In [7]:
books_dir = '../data/raw/text_mining/books'
books = os.listdir(books_dir)
books = [book.split('.pdf')[0] for book in books]
books

['An Introduction To Information Retrieval',
 'Applied Text Analysis with Python-2016',
 'Foundations of Statistical Natural Language Processing - Christopher D. Manning',
 'Natural Language Processing with Python-2009',
 'Natural Language Understanding with Distributed Representation-2017',
 'Representation Learning for Natural Language Processing',
 'Text_Analytics_with_Python']

In [8]:
for doc in processed_docs:
    source_type = 'book' if doc.metadata['docname'] in books else 'article' 
    doc.metadata = doc.metadata | {'source_type': source_type}

In [9]:
from typing import List
import together

together.api_key = token
client = together.Together()

def get_embedding(text: List[str], model: str) -> List[List[float]]:
    text = text.replace("\n", " ")
    outputs = client.embeddings.create(input = [text], model=model)
    return outputs.data[0].embedding

In [11]:
save_directory = '../data/processed/embeddings'

with open(f"{save_directory}/total_embeddings.pickle", 'rb') as f:
    embeddings = pickle.load(f)

In [13]:
len(embeddings)

1907

In [ ]:
n_exceptions = 0

for doc in tqdm(processed_docs[1788:]):
    text = doc.page_content
    
    while n_exceptions < 5:
        try:
            embedding = get_embedding(text, model=model)
            embeddings.append(embedding)
            n_exceptions = 0
            time.sleep(1)
            break
        except Exception as e:
            print(f'Got Exception: {e}')
            n_exceptions += 1
            time.sleep(1)
            
            text = text[:-150]
    else:
        print(f'Got five exceptions! Saving {len(embeddings)} book embeddings to persist directory!')
        break
        
with open(f"{save_directory}/total_embeddings.pickle", 'wb') as f:
    pickle.dump(embeddings, f)

  0%|          | 0/3683 [00:00<?, ?it/s]

Got Exception: 400 Client Error: Bad Request for url: https://api.together.xyz/api/v1/embeddings
Got Exception: 400 Client Error: Bad Request for url: https://api.together.xyz/api/v1/embeddings
Got Exception: 400 Client Error: Bad Request for url: https://api.together.xyz/api/v1/embeddings


In [31]:
print(f'Got five exceptions! Saving {len(book_embeddings)} book embeddings to persist directory!')
with open(f"{save_directory}/book_embeddings.pickle", 'wb') as f:
    pickle.dumps(book_embeddings)

Got five exceptions! Saving 1201 book embeddings to persist directory!


In [43]:
class DummyEmbeddings(Embeddings):
    def __init__(self, emb_mapper):
        self.emb_mapper = emb_mapper
        
    def __call__(self, texts: List[str]) -> Embeddings:
        return self.embed_documents(texts)
    
    def embed_query(self, text: str) -> List[float]:
        return self.emb_mapper[text]
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        outputs = []
        for text in texts:
            outputs.append(self.emb_mapper[text])
        return outputs

In [44]:
vectorstore = Chroma.from_documents(book_docs, DummyEmbeddings(emb_mapper), persist_directory="../data/vectorstores/storage_v0.4")

## Get vectorstore from memory

In [113]:
class DummyEmbeddings(Embeddings):
    def __init__(self, emb_mapper):
        self.emb_mapper = emb_mapper
        
    def __call__(self, texts: List[str]) -> Embeddings:
        return self.embed_documents(texts)
    
    def embed_query(self, text: str) -> List[float]:
        return self.emb_mapper[text]
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        outputs = []
        for text in texts:
            outputs.append(self.emb_mapper[text])
        return outputs

In [114]:
docs = vectorstore.get(include=['embeddings', 'metadatas', 'documents'])

emb_mapper = dict(zip(docs['documents'], docs['embeddings']))
emb = DummyEmbeddings(emb_mapper)

In [116]:
model = 'BAAI/bge-base-en-v1.5'
token = '4fbc8ea1ec756dd6d645a7db75ebb6e8a85cdaf81b09acb2bbe3143a017d11c8'
persist_directory = "../data/vectorstores/storage_v0.4"

embeddings = TogetherEmbeddings(model=model, token=token)

vectorstore = Chroma(persist_directory=persist_directory, embedding_function=emb)

In [117]:
for i, id_ in enumerate(docs['ids']):
    new_metadata = docs['metadatas'][i] | {'source_type': 'book'}
    new_doc = Document(page_content=docs['documents'][i], metadata=new_metadata, embedding=docs['embeddings'][i])
    
    vectorstore.update_document(id_, new_doc)

## Query to storage

In [118]:
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [119]:
def get_relevant_documents(db, query, source=None, n_answers=3):
    docs = db.similarity_search_with_score(query, filter = {'docname': source} if source else None, k=n_answers)
    
    return [doc[0] for doc in docs]
    return docs

def process_prompt(prompt, folder_id):
    url = f'gpt://{folder_id}/yandexgpt/latest'


def process_question(question: str, vectorstore, yandex_folder_id: str, source=None, n_documents=3):
    docs = get_relevant_documents(vectorstore, question, source=source, n_answers=n_documents)
    context = '\n\n'.join([doc.page_content for doc in docs])
    
    TEMPLATE = """
You are an expert in Data Science, NLP and Text Mining. 
You answer the user's questions using the provided context. Below is the user's question, as well as several texts that form the context. Your task is to answer the user's question using only the information from the given texts. If the information contained in the texts is not relevant, you should try to answer it yourself, but in this case, be sure to indicate that there is no information in the context.
You must answer in the same language in which the user asked you the question. For example, if the question was in English, then the answer should also be in English. Russian Russian, then the answer is in Russian.

Users'question: {question}
Texts as context:
{context}
"""
    prompt = TEMPLATE.join(question=question, context=context)

In [120]:
def get_response(question, context, iam_token):
    url = f'gpt://{FOLDER_ID}/yandexgpt/latest'
    
    headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {iam_token}",
                "x-folder-id": f"{FOLDER_ID}"
            }
    
    body = {
      "modelUri": f"gpt://{FOLDER_ID}/yandexgpt/latest",
      "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
      },
      "messages": [
        {
          "role": "system",
          "text": "You are an expert in Data Science, NLP and Text Mining NLP and Text Mining.\nYou answer the user's questions using the provided context. Below is the user's question, as well as several texts that form the context. Your task is to answer the user's question using only the information from the given texts. If the information contained in the texts is not relevant, you should try to answer it yourself, but in this case, be sure to indicate that there is no information in the context.\nYou must answer in the same language in which the user asked you the question. For example, if the question was in English, then the answer should also be in English. If question is in Russian, then the answer is in Russian."
          },
        {
          "role": "user",
          "text": f"Question: {question}\nContext:\n{context}"
        }
      ]
    }
    response = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completion", headers=headers, data=json.dumps(body))
    
    if response.status_code != 200:
        raise Exception(f'generation failed! Status code: {response.status_code}\n{response.text}')
            
    return response.json()['result']['alternatives'][0]['message']['text']

In [121]:
query = 'Explain TF IDF algorithm'
docs = get_relevant_documents(vectorstore, query, source=None, n_answers=3)
context = '\n\n'.join([doc.page_content for doc in docs])

for doc in docs:
    print(doc.metadata)
    print(doc.page_content)
    print('='*100)

{'docname': 'Text_Analytics_with_Python', 'source_type': 'book'}
for this algorithm:
 # define function to compute tfidf weighted averaged word vector for a document
def tfidf_wtd_avg_word_vectors(words, tfidf_vector, tfidf_vocabulary, model, 
num_features):
 weighted averaged word vector representation for a document. We also create a 
corresponding generic function tfidf_weighted_averaged_word_vectorizer() to 
perform TF-IDF weighted averaging of word vectors for a corpus of documents. We 
can see our implemented function in action on our sample corpora using the following 
snippet:
 # get tfidf weights and vocabulary from earlier results and compute result
In : corpus_tfidf = tdidf_features
 ...: vocab = tfidf_vectorizer.vocabulary_
 ...: wt_tfidf_word_vec_features = tfidf_weighted_averaged_word_
vectorizer(corpus=TOKENIZED_CORPUS, tfidf_vectors=corpus_tfidf,
 ...: tfidf_vocabulary=vocab, model=model, 
 ...: print np.round(wt_tfidf_word_vec_features, 3)

 
 
 ]
 # compute avgd word 

In [15]:
print(context[:10000])

for this algorithm:
 # define function to compute tfidf weighted averaged word vector for a document
def tfidf_wtd_avg_word_vectors(words, tfidf_vector, tfidf_vocabulary, model, 
num_features):
 weighted averaged word vector representation for a document. We also create a 
corresponding generic function tfidf_weighted_averaged_word_vectorizer() to 
perform TF-IDF weighted averaging of word vectors for a corpus of documents. We 
can see our implemented function in action on our sample corpora using the following 
snippet:
 # get tfidf weights and vocabulary from earlier results and compute result
In : corpus_tfidf = tdidf_features
 ...: vocab = tfidf_vectorizer.vocabulary_
 ...: wt_tfidf_word_vec_features = tfidf_weighted_averaged_word_
vectorizer(corpus=TOKENIZED_CORPUS, tfidf_vectors=corpus_tfidf,
 ...: tfidf_vocabulary=vocab, model=model, 
 ...: print np.round(wt_tfidf_word_vec_features, 3)

 
 
 ]
 # compute avgd word vector for test new_doc
In : nd_wt_tfidf_word_vec_features = tfid

## Good examples of questions

* Pros and cons of TF-IDF
* Pros of classic methods of machine learning against neural networks
* Main tasks of text analisys?
* Differences of classification and regression
* What is Word2Vec? Differences with FastText?

## Invalid articles and books

* Scalable Text Mining with Sparse Generative Models
* Natural Language Understanding with Distributed Representation-2017

## Выводы

Очень много мэтчей с текстами из книг, кажется, что в книгах находится очень много информации, которая может быть полезной в информационной системе. В статьях чаще всего можно встретить очень продвинутые и узконаправленные темы, нежели информацию интересную пользователю.

Можем попробовать расширить длину контекста для того, чтобы уместить больше информации, макс 8192 токена. \
Модель: https://huggingface.co/BAAI/bge-large-en-v1.5

На следующей итерации будем работать исключительно с книгами по text mining. Добавить более общие книги, например по python и анализу данных в целом. Необходимо научиться работать с формулами. 

In [135]:
from typing import List
import together

together.api_key = token
client = together.Together(api_key=token)

def get_embedding(text: List[str], model: str) -> List[List[float]]:
    text = text.replace("\n", " ")
    outputs = client.embeddings.create(input = [text], model=model)
    return outputs.data[0].embedding

arxiv_embeddings = []
n_exceptions = 0

save_directory = '../data/processed/embeddings'

for doc in tqdm(arxiv_docs):
    while n_exceptions < 5:
        try:
            embedding = get_embedding(doc.page_content, model=model)
            arxiv_embeddings.append(embedding)
            n_exceptions = 0
            time.sleep(1)
            break
        except Exception as e:
            print(f'Got Exception: {e}')
            n_exceptions += 1
    else:
        print(f'Got five exceptions! Saving {len(arxiv_embeddings)} book embeddings to persist directory!')
        break
        
with open(f"{save_directory}/arxiv_embeddings.pickle", 'wb') as f:
    pickle.dumps(arxiv_embeddings)

TypeError: Together.__init__() got an unexpected keyword argument 'api_key'

In [129]:
len(arxiv_docs)

483